In [4]:
import os
import tweepy

In [5]:
import configparser
config = configparser.ConfigParser()
config.read_file(open('dwh.cfg'))

consumer_key= config.get('KEY','consumer_key')
consumer_secret= config.get('KEY','consumer_secret')
access_token= config.get('KEY','access_token')
access_token_secret= config.get('KEY','access_token_secret')

os.environ["GOOGLE_APPLICATION_CREDENTIALS"]=config.get('KEY','json_loc')

auth= tweepy.OAuthHandler(consumer_key,consumer_secret)
auth.set_access_token(access_token,access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)

In [41]:
from google.cloud import vision

client = vision.ImageAnnotatorClient()
image = vision.Image()

def parseImage(imageURL):
    image.source.image_uri = image_url
    response = client.text_detection(image=image)
    string = ""
    for text in response.text_annotations:
        string =  text.description
        return string


In [45]:
import re
def parseString(string):
    val = dict()

    #Date
    match=(re.search("/ ", string))
    i = int(match.end())
    match=(re.search("\)", string))
    j = int(match.start())
    print(string[i:j])
    # val['day'] = string[:string.find(" ")]
    val['month'] = 'April'
    val['year'] = '2021'


    #+ve cases
    i = string.find("Positive Cases", string.find("Positive Cases")+1)
    match = (re.search("Tests Conducted", string))
    j = int(match.start())
    val['positive'] = string[i + len("Positive Cases") + 1 : j - 1]

    #Tests
    match=(re.search("Tests Conducted", string))
    i = int(match.end())
    match=(re.search("Positivity Rate", string))
    j = int(match.start())
    val['tests'] = string[i+1:j-1]

    #Recovered
    match=(re.search("%", string))
    i = int(match.end())
    match=(re.search("Deaths", string))
    j = int(match.start())
    val['recovered'] = string[i+1:j-1]

    #Deaths
    match=(re.search("Deaths", string))
    i = int(match.end())
    match=(re.search("COVID-19 Patient Management", string))
    j = int(match.start())
    val['deaths'] = string[i+1:j-1]

    #Tot Vac
    match=(re.search("Beneficiaries vaccinated in last 24 hours", string))
    i = int(match.end())
    match=(re.search("Beneficiaries vaccinated 1", string))
    j = int(match.start())
    val['vaccinated'] = string[i+1:j-1]

    #1st dose
    match=(re.search("dose in last 24 hours", string))
    i = int(match.end())
    match=(re.search("Beneficiaries vaccinated 2", string))
    j = int(match.start())
    val['first_dose'] = string[i+1:j-1]

    #2nd dose
    pat = "dose in last 24 hours"
    match=(re.search(pat, string))
    pos = string.find(pat)+1
    p1 = string.find(pat, pos+1)
    i = p1 + len(pat)
    match=(re.search("Cumulative beneficiaries vaccinated so far", string))
    j = int(match.start())
    val['second_dose'] = string[i+1:j-1]

    #Active Cases
    match=(re.search("Active Cases", string))
    i = int(match.end())
    match=(re.search("Total Number of Containment Zones as on date", string))
    j = int(match.start())
    val['active_cases'] = string[i+1:j-1]

    #Containment
    match=(re.search("Total Number of Containment Zones as on date: ", string))
    i = int(match.end())
    match=(re.search("Calls received", string))
    j = int(match.start())
    val['zones'] = string[i:j-1]

    print(val)
    return val


In [46]:
def addData(data):
    return "an"

In [47]:
date_since = "2021-04-01"
search_words = "Delhi Health Bulletin (from:CMODelhi)"

tweets = tweepy.Cursor(api.search,
              q=search_words,
              lang="en",
              since=date_since).items(1)

for tweet in tweets:
    media = tweet.entities.get('media', [])
    image_url = media[0]['media_url']
    parseString(parseImage(image_url))

May 5th 2021
{'month': 'April', 'year': '2021', 'positive': '20960', 'tests': '79491', 'recovered': '19209', 'deaths': '311', 'vaccinated': '90365', 'first_dose': '64983', 'second_dose': '25382', 'active_cases': '91859', 'zones': '47704'}
